In [1]:
filename = "day-20-input.txt"

# Part 1
This makes the simplifying assumption that the tiles' border matches are constrained enough such that my solution works. My assumption turned out to be true.

In [2]:
import collections
import itertools
from enum import Enum
from typing import List, Dict, Tuple


class Tile:
    """Represents a tile."""
    
    def __init__(self, tile_id: int, grid: List[List[str]]):
        self.id = tile_id
        self.grid = grid
        
        self.num_rows = len(grid)
        self.num_cols = len(grid[0])

        self.border_top = "".join(grid[0])
        self.border_right = "".join([row[-1] for row in grid])
        self.border_bottom = "".join(grid[-1])
        self.border_left = "".join([row[0] for row in grid])
        self.borders = [
            self.border_top,
            self.border_right,
            self.border_bottom,
            self.border_left,
        ]
        
    def __str__(self) -> str:
        string_grid = []
        
        for row in self.grid:
            string_grid.append("".join(row))
        
        return "\n".join(string_grid)
    
    @classmethod
    def from_raw(cls, raw_tile: str):
        lines = raw_tile.splitlines()
        
        tile_id = int(lines[0].split()[1].split(":")[0])
        
        grid_lines = lines[1:]
        grid = []
        for line in grid_lines:
            grid.append(list(line))
        
        return cls(tile_id, grid)
    
    def rotate(self) -> None:
        """Rotates 90 degrees clockwise."""
        new_grid = [[" " for c in range(self.num_cols)] 
                    for r in range(self.num_rows)]
        
        for row_idx, row in enumerate(self.grid):
            for col_idx, char in enumerate(row):
                new_grid[col_idx][self.num_rows-row_idx-1] = char
        
        self.__init__(self.id, new_grid)
    
    def flip(self) -> None:
        """Flips on y-axis."""
        new_grid = [[" " for c in range(self.num_cols)] 
                    for r in range(self.num_rows)]
        
        for row_idx, row in enumerate(self.grid):
            for col_idx, char in enumerate(row):
                new_grid[row_idx][self.num_cols-col_idx-1] = char
                
        self.__init__(self.id, new_grid)

In [3]:
with open(filename) as file:
    tiles = {}
    for raw_tile in file.read().strip().split("\n\n"):
        tile = Tile.from_raw(raw_tile)
        tiles[tile.id] = tile
        

# border -> tile ids
border_to_ids = collections.defaultdict(set)

for tile in tiles.values():
    for border in tile.borders:
        border_to_ids[border].add(tile.id)
        border_to_ids[border[::-1]].add(tile.id)

# tile id -> ids of other tiles who have a matching border
tile_id_to_partners = collections.defaultdict(set)
for border, tile_ids in border_to_ids.items():
    for tile_id_1, tile_id_2 in itertools.permutations(tile_ids, 2):
        tile_id_to_partners[tile_id_1].add(tile_id_2)

# The corner tiles will only have 2 partner tiles.
corner_product = 1
corner_tile_ids = []
for tile_id, partners in tile_id_to_partners.items():
    if len(partners) == 2:
        corner_product *= tile_id
        corner_tile_ids.append(tile_id)
        
print("The product of the IDs of the four corner tiles is:", corner_product)

The product of the IDs of the four corner tiles is: 51214443014783


# Part 2
This solution assumes that:
* The tiles are arranged into a square. (They are.)
* The tiles with matching borders are constrained such that each tile has a definitive set of neighbors. That is, a corner tile will have 2 neighbors, a non-corner edge tile will have 3 neighbors, and a non-edge tile will have 4 neighbors. (This is all true; proving this in the notebook will be left as an exercise to the reader. 😉)

Sorry, this is not the cleanest. But doesn't matter, solved the problem! 🤘

## Arrange Tiles

In [4]:
class Direction(Enum):
    TOP = 0
    RIGHT = 1
    BOTTOM = 2
    LEFT = 3


class Arrangement:
    """Represents a square arrangement of tiles."""
    
    def __init__(self,
                 tiles: Dict[int, Tile],
                 tile_id_to_partners: Dict[int, int]):        
        self.tiles = tiles
        self.tile_id_to_partners = tile_id_to_partners
        
        side_length = int(len(tiles) ** 0.5)
        self.tile_grid = [[None for c in range(side_length)] for r in range(side_length)]
        self.tile_id_to_coord = {}  # tile_id -> (row, col) coordinate in grid
        
        # Keep track of next tiles to place
        self.next_tiles = set()
    
    def place_first_corner_tile(self, tile_id) -> None:
        assert len(self.tile_id_to_coord) == 0
        
        tile = self.tiles[tile_id]
        partner_ids = list(self.tile_id_to_partners[tile_id])
        # We can assume this because we'll later flip/rotate the Arrangement
        # as necessary to form the image with sea monsters.
        right_neighbor = self.tiles[partner_ids[1]]
        bottom_neighbor = self.tiles[partner_ids[0]]
        
        # Get both unflipped and flipped versions of neighbors' borders
        right_neighbor_borders = []
        right_neighbor_borders.extend(right_neighbor.borders)
        right_neighbor_borders.extend([border[::-1] for border in right_neighbor.borders])

        bottom_neighbor_borders = []
        bottom_neighbor_borders.extend(bottom_neighbor.borders)
        bottom_neighbor_borders.extend([border[::-1] for border in bottom_neighbor.borders])
        
        # Rotate tile until it's in the correct position.
        for i in range(8):
            if i == 4:
                # We've rotated a full 360, time to flip.
                tile.flip()
            if tile.border_right in right_neighbor_borders and tile.border_bottom in bottom_neighbor_borders:
                break
            tile.rotate()
        
        # bookkeeping
        self.tile_id_to_coord[tile_id] = (0, 0)
        self.tile_grid[0][0] = tile
        
        # Return its neighbors as ready to be placed
        self.next_tiles.add((right_neighbor.id, (0, 1)))
        self.next_tiles.add((bottom_neighbor.id, (1, 0)))

    def place_tile(self, tile_id: int, row_idx: int, col_idx: int) -> None:
        """Places tile. Returns information on which tiles can be placed next."""
        
        if tile_id in self.tile_id_to_coord:
            raise Exception("We've already placed this tile!")
        
        tile = self.tiles[tile_id]
        
        partner_ids = self.tile_id_to_partners[tile_id]
        for partner_id in partner_ids:
            if partner_id in self.tile_id_to_coord:
                
                partner_tile = self.tiles[partner_id]
                
                partner_r, partner_c = self.tile_id_to_coord[partner_id]
                if partner_r - row_idx == 1:
                    direction = Direction.TOP
                elif partner_c - col_idx == -1:
                    direction = Direction.RIGHT
                elif partner_r - row_idx == -1:
                    direction = Direction.BOTTOM
                elif partner_c - col_idx == 1:
                    direction = Direction.LEFT
                else:
                    Exception("I didn't expect to hit here!")
                
                self.rotate_neighbor_into_place(partner_tile, tile, direction)
                break
                
        # Bookkeeping
        self.tile_id_to_coord[tile_id] = (row_idx, col_idx)
        self.tile_grid[row_idx][col_idx] = self.tiles[tile_id]
        
        # Figure out the next tiles to visit
        # Since we're filling out the grid from the top left, we are only adding
        # in tiles to the right and to the bottom.
        for partner_id in partner_ids:
            if partner_id not in self.tile_id_to_coord:
                partner_tile = self.tiles[partner_id]
                potential_borders = []
                potential_borders.extend(partner_tile.borders)
                potential_borders.extend([border[::-1] for border in partner_tile.borders])

                # Case: neighbor tile is to the right
                if tile.border_right in potential_borders:
                    self.next_tiles.add((partner_id, (row_idx, col_idx+1)))
                    
                # Case: neighbor tile is to the bottom
                if tile.border_bottom in potential_borders:
                    self.next_tiles.add((partner_id, (row_idx+1, col_idx)))
        
        
    @staticmethod
    def rotate_neighbor_into_place(tile: Tile, neighbor: Tile, direction: Direction) -> None:
        """
        `tile` is set in place. Rotate *and flip* `neighbor` until it fits with `tile`.
        Sorry, function is not named well, and I don't feel like renaming it.
        `direction` is the direction from `tile` to `neighbor`.
        """
        
        for i in range(8):
            if i == 4:
                # We've rotated neighbor a full 360, time to flip neighbor.
                neighbor.flip()
            
            if tile.borders[direction.value] == neighbor.borders[(direction.value + 2) % 4]:
                break
            neighbor.rotate()

In [5]:
a = Arrangement(tiles, tile_id_to_partners)

a.place_first_corner_tile(corner_tile_ids[0])

while a.next_tiles:
    tile_id, (row_idx, col_idx) = a.next_tiles.pop()
    a.place_tile(tile_id, row_idx, col_idx)

## Find Sea Monsters

In [6]:
def make_sea_monster() -> List[List[int]]:
    sea_monster_lines = """                  # 
#    ##    ##    ###
 #  #  #  #  #  #   """.splitlines()
    
    locations = []
    for line in sea_monster_lines:
        line_locs = []
        for idx, char in enumerate(line):
            if char == "#":
                line_locs.append(idx)
        locations.append(line_locs)

    return locations

In [7]:
class Image:
    
    def __init__(self, arrangement: Arrangement):
        self.grid = []  # type: List[List[str]]
        
        for tile_row in arrangement.tile_grid:
            # We will be added this to self.grid
            component = [[] for i in range(tile_row[0].num_rows-2)]
            
            for tile in tile_row:
                for ridx, row in enumerate(tile.grid[1:-1]):
                    
                    component[ridx].extend(row[1:-1])
            self.grid.extend(component)
        
        self.num_rows = len(self.grid)
        self.num_cols = len(self.grid[0])
        
        self.sea_monster = make_sea_monster()
        self.sea_monster_height = 3
        self.sea_monster_length = 20
            
    def __str__(self) -> str:
        string_grid = []
        
        for row in self.grid:
            string_grid.append("".join(row))
        
        return "\n".join(string_grid)

    def rotate(self) -> None:
        """Rotates 90 degrees clockwise. Yes, I'm repeating myself."""
        new_grid = [[" " for c in range(self.num_cols)] 
                    for r in range(self.num_rows)]
        
        for row_idx, row in enumerate(self.grid):
            for col_idx, char in enumerate(row):
                new_grid[col_idx][self.num_rows-row_idx-1] = char
        
        self.grid = new_grid
    
    def flip(self) -> None:
        """Flips on y-axis. Yes, I'm repeating myself."""
        new_grid = [[" " for c in range(self.num_cols)] 
                    for r in range(self.num_rows)]
        
        for row_idx, row in enumerate(self.grid):
            for col_idx, char in enumerate(row):
                new_grid[row_idx][self.num_cols-col_idx-1] = char
                
        self.grid = new_grid
        
    def find_sea_monsters(self):
        
        for i in range(8):
            if i == 4:
                # We've rotated the image a full 360, time to flip.
                self.flip()
            
            if self._find_sea_monster_helper():
                break
            self.rotate()
            
    def _find_sea_monster_helper(self) -> bool:
        
        found_monster = False
        
        for row_idx in range(self.num_rows-self.sea_monster_height+1):
            for col_idx in range(self.num_cols-self.sea_monster_length+1):
                if self._is_sea_monster_here(row_idx, col_idx):
                    found_monster = True
                    
        return found_monster
                        
    def _is_sea_monster_here(self, row_idx, col_idx):
        """Also writes in the monster."""
        
        for i, locs in enumerate(self.sea_monster):
            for loc in locs:
                if self.grid[row_idx+i][col_idx+loc] != "#":
                    return False
        
        # Write in the monster
        for i, locs in enumerate(self.sea_monster):
            for loc in locs:
                self.grid[row_idx+i][col_idx+loc] = "O"
        
        return True

    def water_roughness(self) -> int:
        count = 0
        for row in self.grid:
            for char in row:
                if char == "#":
                    count += 1
        return count

In [8]:
image = Image(a)
image.find_sea_monsters()

print("Roughness of water is:", image.water_roughness())

Roughness of water is: 2065


## Here be sea monsters:

In [9]:
print(image)

...#........#.......#.......#.................#..##....#.......##.........#...........#..#..#...
#.....##...##....#.......#.....#.......###.......####...##..........##.#........#.........#.....
........#.......##.....#.....#.#..#.#....#....#....#...#.#.#.............#.........#.#......#..#
..#...#........#....................#....#..#.#...#.#.#........#...#..#.#...##......#..#.....#.#
#.....#...#..#..#...#.#.#........#..###..........#O#.............#..#..#.#.....O#.##..#.........
.......#.#...#.....#...#.#......O##..OO..#.OO....OOO.#...#...O.#..OO....OO...#OOO.##.#.........#
...................###..##.#.##.#O..O#.O.#O..O#.O#.#....#.....O..O..O..O..O..O....#.##.....#..#.
........#.##....#......#...#.......###.....#.....#..#.....#..##...#..#.#......#....#........#...
#.##...#........#.....#...#...#....#..................##....#..#....#.#......#.#.#.......##.....
....#.#..#...#......#...........#.............#.##..#..#...#.#....#..#...##...##...#.........#..
..#.....#.#..#....#...#....#..